In [ ]:
# Cell 1: Setup
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.email_detector.detector import EmailPhishingDetector
from src.utils.data_loader import create_sample_email_data
from src.utils.visualization import plot_confusion_matrix

%matplotlib inline
print("✅ Setup complete!")


In [ ]:
# Cell 2: Load and Prepare Data
emails_df = create_sample_email_data()
labels = emails_df['label'].tolist()

print(f"📧 Total emails: {len(emails_df)}")
print(f"🔴 Phishing: {sum(labels)}")
print(f"🟢 Safe: {len(labels) - sum(labels)}")

In [ ]:
# Cell 3: Train Email Detector
detector = EmailPhishingDetector()

print("🚀 Training email phishing detector...")
features = detector.train(emails_df, labels)

print(f"\n✅ Training complete!")
print(f"📊 Total features: {len(detector.feature_names)}")


In [ ]:
# Cell 4: Feature Importance
feature_importance = detector.model.feature_importances_
top_indices = np.argsort(feature_importance)[-15:][::-1]

print("🎯 Top 15 Most Important Features:")
for i, idx in enumerate(top_indices, 1):
    print(f"{i}. {detector.feature_names[idx]}: {feature_importance[idx]:.4f}")

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_indices)), feature_importance[top_indices])
plt.yticks(range(len(top_indices)), [detector.feature_names[i] for i in top_indices])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances')
plt.tight_layout()
plt.show()

In [ ]:
# Cell 5: Test Predictions
test_emails = [
    {
        'text': "URGENT! Your account will be suspended! Click here NOW!",
        'sender': "security@fake-bank.com",
        'subject': "URGENT ACTION REQUIRED"
    },
    {
        'text': "Hi team, please review the attached document for our meeting.",
        'sender': "colleague@company.com",
        'subject': "Meeting Document"
    }
]

print("🧪 Testing predictions:\n")

for i, email in enumerate(test_emails, 1):
    result = detector.predict_with_explanation(
        email['text'], 
        email['sender'], 
        email['subject']
    )
    
    print(f"Test Email {i}:")
    print(f"  Subject: {email['subject']}")
    print(f"  Prediction: {result['prediction']}")
    print(f"  Confidence: {result['confidence']:.1f}%")
    print(f"  Risk Factors: {len(result['risk_factors'])}")
    for factor in result['risk_factors'][:3]:
        print(f"    • {factor}")
    print()

In [ ]:
# Cell 6: XAI - LIME Explanation
print("🔍 Explainable AI Analysis:")

result = detector.predict_with_explanation(
    "URGENT! Your PayPal account will be suspended! Verify now!",
    "security@paypal-fake.com",
    "Account Suspension Warning"
)

print(f"\nPrediction: {result['prediction']}")
print(f"Confidence: {result['confidence']:.1f}%")

print("\n💡 Top LIME Explanations:")
for feature, weight in result['lime_explanation'][:10]:
    direction = "↗️ Increases" if weight > 0 else "↘️ Decreases"
    print(f"  {feature}: {direction} phishing risk ({weight:.3f})")


In [ ]:
# Cell 7: Save Model
import os
os.makedirs('../models', exist_ok=True)

model_path = '../models/email_detector_v1.pkl'
detector.save_model(model_path)

print(f"✅ Model saved to {model_path}")
print("\n➡️ Next: Web log analysis in 03_web_log_analysis.ipynb")